In [1]:
import torch
import evaluate
import numpy as np

from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorWithPadding, DataCollatorForSeq2Seq

In [2]:
save_model = './lora_trained/lora-flan-t5-rte/'
base_model = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(base_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)
dataset = load_dataset('glue','mnli')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

c:\Users\incognito\miniconda3\envs\a1_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


cuda


In [3]:
total_params = sum(p.numel() for p in model.parameters())
print(total_params * 4)
for key ,value in enumerate(dataset):
    print(key, value)

990311424
0 train
1 validation_matched
2 validation_mismatched
3 test_matched
4 test_mismatched


In [11]:
def data_preprocessing(batch):
    inputs = [f"premise: {premise} hypothesis: {hypothesis}" for premise, hypothesis in zip(batch['premise'], batch['hypothesis'])]
    labels = batch['label']

    label_map = {0: "neutral", 1: "entailment", 2: "contradiction"}
    processed_labels = [label_map[label] if label in label_map else None for label in labels]

    valid_inputs = [input_text for input_text, label in zip(inputs, processed_labels) if label is not None]
    valid_labels = [label for label in processed_labels if label is not None]

    model_inputs = tokenizer(valid_inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        tokenized_labels = tokenizer(valid_labels, max_length=16, truncation=True, padding="max_length")

    model_inputs["labels"] = tokenized_labels["input_ids"]
    return model_inputs

def process_dataset_in_batches(dataset, batch_size=32):
    processed_dataset = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i + batch_size]
        processed_batch = data_preprocessing(batch)
        
        processed_dataset["input_ids"].extend(processed_batch["input_ids"])
        processed_dataset["attention_mask"].extend(processed_batch["attention_mask"])
        processed_dataset["labels"].extend(processed_batch["labels"])

    return processed_dataset

train_processed = process_dataset_in_batches(dataset["train"], batch_size=32)

validation_processed = process_dataset_in_batches(dataset["validation_matched"], batch_size=32)

train_dataset = Dataset.from_dict({
    "input_ids": train_processed["input_ids"],
    "attention_mask": train_processed["attention_mask"],
    "labels": train_processed["labels"]
})

validation_dataset = Dataset.from_dict({
    "input_ids": validation_processed["input_ids"],
    "attention_mask": validation_processed["attention_mask"],
    "labels": validation_processed["labels"]
})

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(validation_dataset)}")

c:\Users\incognito\miniconda3\envs\a1_env\Lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Training dataset size: 392702
Validation dataset size: 9815


In [12]:
print(train_dataset[0])

{'input_ids': [3, 17398, 10, 16688, 76, 1427, 3022, 5210, 635, 53, 65, 192, 1857, 8393, 3, 18, 556, 11, 25055, 5, 22455, 10, 6246, 11, 25055, 33, 125, 143, 3022, 5210, 635, 53, 161, 5, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [13]:
accuracy_metric = evaluate.load("accuracy")
def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)  # Replace ignored index (-100)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    accuracy = sum([pred == label for pred, label in zip(decoded_preds, decoded_labels)]) / len(decoded_preds)

    return {
        "accuracy": accuracy
    }

In [14]:
# data_collator = DataCollatorWithPadding(tokenizer, model=model)

lora_config = LoraConfig(
    r=8, 
    lora_alpha=32,  
    target_modules=["q", "v"],  
    lora_dropout=0.1,  
    bias="none"
)


lora_model = get_peft_model(model, lora_config)
lora_model.to(device)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=lora_model)
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",  
    evaluation_strategy="epoch", 
    learning_rate=1e-5,
    per_device_train_batch_size=12,  
    per_device_eval_batch_size=12, 
    weight_decay=0.01, 
    save_total_limit=3,
    num_train_epochs=3
)



# Define Trainer
trainer = Seq2SeqTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,  # The preprocessed training dataset
    eval_dataset=validation_dataset,  # The preprocessed validation dataset
    # data_collator=data_collator,  # Handles batch padding
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Fine-tune the LoRA model
# uncommnet this to train
# make sure to change model_name from the top code shell to base model for retraining.
trainer.train()

c:\Users\incognito\miniconda3\envs\a1_env\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/98178 [00:00<?, ?it/s]

In [ ]:
train_dataset

In [ ]:

results = trainer.evaluate()
print(results)


In [ ]:
model.save_pretrained(save_model)
tokenizer.save_pretrained(save_model)